In [4]:
import wetter_api as wt
import waldbrand
import model_eval as me
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

In [9]:
parameter_list = [
    wt.DwdObservationParameter.MONTHLY.PRECIPITATION_HEIGHT,
    wt.DwdObservationParameter.MONTHLY.TEMPERATURE_AIR_MAX_200,
    wt.DwdObservationParameter.MONTHLY.TEMPERATURE_AIR_MIN_200,
    wt.DwdObservationParameter.MONTHLY.SUNSHINE_DURATION,
]
# Dictionary with short names as keys and full names as values
shortname_to_state = {
    "BW": "Baden-Wuerttemberg",
    "BY": "Bayern",
    "BE": "Berlin",
    "BB": "Brandenburg",
    "HB": "Bremen",
    "HH": "Hamburg",
    "HE": "Hessen",
    "MV": "Mecklenburg-Vorpommern",
    "NI": "Niedersachsen",
    "NW": "Nordrhein-Westfalen",
    "RP": "Rheinland-Pfalz",
    "SL": "Saarland",
    "SN": "Sachsen",
    "ST": "Sachsen-Anhalt",
    "SH": "Schleswig-Holstein",
    "TH": "Thueringen",
    "FB": "Frueheres Bundesgebiet",
    "NL": "Neue Laender",
    "DE": "Deutschland",
}

# Example usage
shortname = "BB"
state = shortname_to_state.get(shortname)

In [10]:
merged_data = me.prepare_data(parameter_list, state)
str_parameter_list = [str(param) for param in parameter_list]
feature_list = ["Year", "Month"] + str_parameter_list
model_results = me.prepare_data_for_ml(merged_data, feature_list)

In [11]:
for model_name, metrics in model_results.items():
    print(f"{model_name} - MSE: {metrics['mse']}, R2: {metrics['r2_score']}")
    if model_name == "Linear Regression":
        print("Coefficients:", metrics["model"].coef_)

Linear Regression - MSE: 761.219528786925, R2: 0.5687460110869479
Coefficients: [-9.00823070e-01 -8.66870402e-02 -2.19062607e-01  5.66869549e-01
 -6.31460307e-01  1.07738193e-04]
Ridge Regression - MSE: 761.2114866258283, R2: 0.5687505672155301
Lasso Regression - MSE: 756.5766596319872, R2: 0.5713763375136215
Decision Tree Regression - MSE: 589.1323529411765, R2: 0.6662386242133174
Random Forest Regression - MSE: 480.39583529411766, R2: 0.7278411648766356
Support Vector Regression - MSE: 1024.1472076272546, R2: 0.41978949327895443
Gradient Boosting Regression - MSE: 559.7627922112109, R2: 0.6828773726143282
